In [1]:
import pyanitools as pyt
import numpy as np
import os

from sklearn import linear_model

import hdnntools as hdt

/usr/local/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [10]:
datadir = '/home/jsmith48/scratch/ccsd_extrapolation/h5files_holdout_split/trainset/delta/'
sae_out = '/home/jsmith48/scratch/ccsd_extrapolation/learning_cases/delta_DZ_retrain/delta_dft_1/sae_linfit.dat'
smap = {'H':0,'C':1,'N':2,'O':3}
Na = len(smap)

In [3]:
files = os.listdir(datadir)
np.random.shuffle(files)

In [4]:
X = []
y = []
for f in files[0:20]:
    print(f)
    adl = pyt.anidataloader(datadir+f)
    for data in adl:
        #print(data['path'])
        S = data['species']
        E = data['energies']
        unique, counts = np.unique(S, return_counts=True)
        x = np.zeros(Na, dtype=np.float64)
        for u,c in zip(unique,counts):
            x[smap[u]]=c
            
        for e in E:
            X.append(np.array(x))
            y.append(np.array(e))

ccsd_combined_delta.h5


In [5]:
X = np.array(X)
y = np.array(y).reshape(-1,1)
print(X.shape)
print(y.shape)

(313054, 4)
(313054, 1)


In [6]:
lin = linear_model.LinearRegression(fit_intercept=False)

In [7]:
lin.fit(X,y)

LinearRegression(copy_X=True, fit_intercept=False, n_jobs=1, normalize=False)

In [8]:
coef = lin.coef_
print(coef)

[[-0.00276818  0.04427959  0.03715691  0.02888213]]


In [11]:
sae = open(sae_out,'w')
for i,c in enumerate(coef[0]):
    sae.write(next(key for key, value in smap.items() if value == i)+','+str(i)+'='+str(c)+'\n')

sae.close()